In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os
ProjDIR = "/home/jw3514/Work/ASD_Circuits_CellType/" # Change to your project directory
sys.path.insert(1, f'{ProjDIR}/src/')
from ASD_Circuits import *

try:
    os.chdir(f"{ProjDIR}/notebook_rebuttal/")
    print(f"Current working directory: {os.getcwd()}")
except FileNotFoundError as e:
    print(f"Error: Could not change directory - {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

HGNC, ENSID2Entrez, GeneSymbol2Entrez, Entrez2Symbol = LoadGeneINFO()

In [ ]:
# Load config file
with open("../config/config.yaml", "r") as f:
    config = yaml.safe_load(f)

expr_matrix_path = config["analysis_types"]["STR_ISH"]["expr_matrix"]
STR_BiasMat = pd.read_parquet(f"../{expr_matrix_path}")
Anno = STR2Region()

In [ ]:
df = pd.read_excel("/home/jw3514/Work/data/DDD/41586_2020_2832_MOESM4_ESM.xlsx")
df = df.sort_values("denovoWEST_p_full")
hc_df = df[df["denovoWEST_p_full"]<=0.05/18762]
entrez_ids = [int(GeneSymbol2Entrez.get(x, -1)) for x in hc_df["symbol"].values]
hc_df["EntrezID"] = entrez_ids
hc_df.shape

In [ ]:
hc_df.head(5)

In [ ]:
hc_df["AutismMerged_LoF"] = (
    df.loc[hc_df.index, "frameshift_variant"].fillna(0)
    + df.loc[hc_df.index, "splice_acceptor_variant"].fillna(0)
    + df.loc[hc_df.index, "splice_donor_variant"].fillna(0)
    + df.loc[hc_df.index, "stop_gained"].fillna(0)
    + df.loc[hc_df.index, "stop_lost"].fillna(0)
).astype(int).clip(lower=0)

hc_df["AutismMerged_Dmis_REVEL0.5"] = df.loc[hc_df.index, "missense_variant"].fillna(0).astype(int).clip(lower=0)

hc_df = hc_df[["EntrezID", "symbol", "AutismMerged_LoF", "AutismMerged_Dmis_REVEL0.5"]]
hc_df = hc_df.set_index("EntrezID", drop=False)

In [ ]:
# Exclude ASD genes from hc_df before bootstrap
ASD_GW = Fil2Dict(ProjDIR+"dat/Genetics/GeneWeights_DN/Spark_Meta_EWS.GeneWeight.DN.gw")
ASD_GENES = list(ASD_GW.keys())
print(f"Total genes in hc_df before excluding ASD: {len(hc_df)}")
print(f"Number of ASD genes to exclude: {len(ASD_GENES)}")

# Filter out ASD genes
hc_df = hc_df[~hc_df["EntrezID"].isin(ASD_GENES)]
print(f"Total genes in hc_df after excluding ASD: {len(hc_df)}")
print(f"Excluded {len(ASD_GENES)} ASD genes")


In [ ]:
hc_df.head(5)

In [ ]:
# DDD_hc_GW = Aggregate_Gene_Weights_NDD(hc_df, out="../dat/GeneWeights/DDD.hc.gw.csv")
# NDD_top61_DF = hc_df.head(61)
# DDD_top61_GW = Aggregate_Gene_Weights_NDD(NDD_top61_DF, out="../dat/GeneWeights/DDD.top61.gw.csv")
#Dict2Fil(DDD_top61_GW, "../dat/GeneWeights/DDD.top61.gw.csv")

In [ ]:
def bootstrap_gene_mutations(
    df,
    n_boot=10,
    weighted=True,
    lof_col="AutismMerged_LoF",
    dmis_col="AutismMerged_Dmis_REVEL0.5",
    rng=None,
):
    """
    Bootstrap mutation counts at the mutation level, preserving gene identity
    and total mutation load. Supports weighted (mutation-rate) or uniform resampling.

    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe with gene-level LOF and Dmis mutation counts.
    n_boot : int, optional
        Number of bootstrap replicates (default = 10).
    weighted : bool, optional
        If True, mutations are resampled with probability proportional to observed counts.
        If False, each gene has equal probability of receiving any mutation.
    lof_col, dmis_col : str
        Column names for LOF and Dmis counts.
    rng : np.random.Generator, optional
        Numpy random generator for reproducibility.

    Returns
    -------
    boot_DFs : list of pd.DataFrame
        List of bootstrapped dataframes with resampled mutation counts and 'bootstrap_iter' column.
    """
    if rng is None:
        rng = np.random.default_rng(42)

    n = len(df)
    boot_DFs = []

    # Ensure integer non-negative mutation counts
    # lof = (df["frameshift_variant"] + df["splice_acceptor_variant"] + df["splice_donor_variant"] + df["stop_gained"] + df["stop_lost"]).astype(int).clip(lower=0)
    # dmis = df["missense_variant"].astype(int).clip(lower=0) 

    lof = df[lof_col].astype(int).clip(lower=0)
    dmis = df[dmis_col].astype(int).clip(lower=0)

    total_lof = lof.sum()
    total_dmis = dmis.sum()

    # Probability vectors for mutation assignment
    if weighted:
        # Weighted by observed mutation burden per gene
        p_lof = lof / total_lof if total_lof > 0 else np.ones(n) / n
        p_dmis = dmis / total_dmis if total_dmis > 0 else np.ones(n) / n
    else:
        # Uniform: every gene equally likely
        p_lof = np.ones(n) / n
        p_dmis = np.ones(n) / n

    for i in range(1, n_boot + 1):
        # Draw total_lof mutation events, assign to genes
        new_lof_counts = np.bincount(
            rng.choice(n, size=total_lof, replace=True, p=p_lof),
            minlength=n
        )
        new_dmis_counts = np.bincount(
            rng.choice(n, size=total_dmis, replace=True, p=p_dmis),
            minlength=n
        )

        # Create bootstrap replicate
        df_boot = df.copy().reset_index(drop=True)
        df_boot[lof_col] = new_lof_counts
        df_boot[dmis_col] = new_dmis_counts
        df_boot["bootstrap_iter"] = i
        df_boot["bootstrap_type"] = "weighted" if weighted else "uniform"
        boot_DFs.append(df_boot)

    return boot_DFs

In [ ]:
boot_DFs_weights = bootstrap_gene_mutations(hc_df, 1000, weighted=True, lof_col="AutismMerged_LoF", dmis_col="AutismMerged_Dmis_REVEL0.5")

In [ ]:
boot_DFs_weights[0]

In [ ]:
def Aggregate_Gene_Weights_NDD2(MutFil, usepLI=False, Bmis=False, out=None):
    gene2MutN = {}
    for i, row in MutFil.iterrows():
        try:
            g = int(row["EntrezID"])
        except:
            print(g, "Error converting Entrez ID")

        nLGD = row["AutismMerged_LoF"] 
        nMis = row["AutismMerged_Dmis_REVEL0.5"] 

        gene2MutN[g] = nLGD * 0.347 + nMis * 0.194
    if out != None:
        writer = csv.writer(open(out, 'wt'))
        for k,v in sorted(gene2MutN.items(), key=lambda x:x[1], reverse=True):
           writer.writerow([k,v]) 
    return gene2MutN

In [ ]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing as mp

def process_bootstrap_iter(args):
    """Worker function to process a single bootstrap iteration"""
    i, DF, save_dir, str_bias_mat = args
    boot_gw = Aggregate_Gene_Weights_NDD2(DF)
    boot_bias = MouseSTR_AvgZ_Weighted(str_bias_mat, boot_gw)
    boot_bias.to_csv(os.path.join(save_dir, f"DDD_ExomeWide.GeneWeight.boot{i}.csv"))
    return i, boot_bias

save_dir = "../results/Bootstrap_bias/DDD_ExomeWide/Weighted_Resampling"
os.makedirs(save_dir, exist_ok=True)

# Prepare arguments for parallel processing
n_workers = mp.cpu_count()  # Use all available CPU cores
args_list = [(i, DF, save_dir, STR_BiasMat) for i, DF in enumerate(boot_DFs_weights)]

# Process in parallel
boot_bias_list_weights = [None] * len(boot_DFs_weights)  # Pre-allocate list to maintain order

with ProcessPoolExecutor(max_workers=n_workers) as executor:
    # Submit all tasks
    future_to_idx = {executor.submit(process_bootstrap_iter, args): args[0] for args in args_list}
    
    # Collect results as they complete
    for future in as_completed(future_to_idx):
        i, boot_bias = future.result()
        boot_bias_list_weights[i] = boot_bias

In [ ]:
## Bootstraped bias residue

from sklearn.linear_model import LinearRegression

def fit_structure_bias_linear_model(merged_data, metric='EFFECT', suffixes=('_1', '_2')):
    """Fit linear regression model and compute residuals"""
    X = merged_data[f'{metric}{suffixes[1]}'].values.reshape(-1, 1)
    y = merged_data[f'{metric}{suffixes[0]}'].values

    model = LinearRegression()
    model.fit(X, y)
    y_pred = model.predict(X)
    residuals = y - y_pred

    results_df = merged_data.copy()
    results_df['predicted'] = y_pred
    results_df['residual'] = residuals

    return results_df

def merge_str_bias_datasets(dataset1, dataset2, suffixes=('_1', '_2')):
    """
    Merge two structure bias datasets for comparison and compute residuals.
    """
    # Select relevant columns
    dataset1_cols = ['Rank', 'EFFECT', 'Region'] if 'Region' in dataset1.columns else ['Rank', 'EFFECT']
    dataset2_cols = ['Rank', 'EFFECT']
    
    # Merge the datasets on structure names
    merged_data = pd.merge(dataset1[dataset1_cols], dataset2[dataset2_cols], 
                          left_index=True, right_index=True, suffixes=suffixes)

    # Calculate differences
    merged_data[f'DIFF_Rank'] = merged_data[f'Rank{suffixes[0]}'] - merged_data[f'Rank{suffixes[1]}']
    merged_data[f'ABS_DIFF_Rank'] = np.abs(merged_data[f'DIFF_Rank'])
    
    merged_data[f'DIFF_EFFECT'] = merged_data[f'EFFECT{suffixes[0]}'] - merged_data[f'EFFECT{suffixes[1]}']
    merged_data[f'ABS_DIFF_EFFECT'] = np.abs(merged_data[f'DIFF_EFFECT'])

    # Sort by absolute difference in EFFECT
    merged_data = merged_data.sort_values('ABS_DIFF_EFFECT', ascending=False)
    
    # Fit linear model and compute residuals
    merged_data = fit_structure_bias_linear_model(merged_data, metric='EFFECT', suffixes=suffixes)
    
    return merged_data

In [ ]:
# Load observed ASD bias
Spark_ASD_STR_Bias = pd.read_csv("../dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.FDR.csv", index_col=0)
Spark_ASD_STR_Bias["Region"] = Spark_ASD_STR_Bias["REGION"]

# Load bootstrap results (if already computed, otherwise use boot_bias_list_weights)
ASD_DIR = "../results/Bootstrap_bias/Spark_ExomeWide/Weighted_Resampling"
DDD_DIR = "../results/Bootstrap_bias/DDD_ExomeWide/Weighted_Resampling"

# Use boot_bias_list_weights if available, otherwise load from files
if 'boot_bias_list_weights' in locals() and len(boot_bias_list_weights) > 0:
    DDD_Boots = boot_bias_list_weights
else:
    DDD_Boots = [pd.read_csv(f"{DDD_DIR}/DDD_ExomeWide.GeneWeight.boot{i}.csv", index_col=0) for i in range(1000)]

# Load ASD boots if available
if os.path.exists(ASD_DIR):
    ASD_Boots = [pd.read_csv(f"{ASD_DIR}/Spark_ExomeWide.GeneWeight.boot{i}.csv", index_col=0) for i in range(1000)]
else:
    print(f"Warning: ASD bootstrap directory not found at {ASD_DIR}")
    ASD_Boots = None

In [ ]:
# Compute residues for each bootstrap iteration (parallelized)
print("Computing residues for each bootstrap iteration...")

def compute_residuals_for_boot(args):
    """Worker function to compute residuals for a single bootstrap iteration"""
    i, ddd_boot, asd_bias = args
    merged = merge_str_bias_datasets(asd_bias, ddd_boot, suffixes=('_ASD', '_DD'))
    # Return index and residuals as a dictionary
    return i, merged['residual'].to_dict()

# Prepare arguments for parallel processing
n_workers = mp.cpu_count()
args_list = [(i, ddd_boot, Spark_ASD_STR_Bias) for i, ddd_boot in enumerate(DDD_Boots)]

# Process in parallel
print(f"Using {n_workers} workers to compute residues...")
residual_results = [None] * len(DDD_Boots)

with ProcessPoolExecutor(max_workers=n_workers) as executor:
    future_to_idx = {executor.submit(compute_residuals_for_boot, args): args[0] for args in args_list}
    
    completed = 0
    for future in as_completed(future_to_idx):
        i, residuals_dict = future.result()
        residual_results[i] = residuals_dict
        completed += 1
        if completed % 100 == 0:
            print(f"Completed {completed}/{len(DDD_Boots)} bootstrap iterations")

# Collect all residues for each structure
structure_residuals = {}  # {structure_name: [residual1, residual2, ...]}
for residuals_dict in residual_results:
    for structure, residual in residuals_dict.items():
        if structure not in structure_residuals:
            structure_residuals[structure] = []
        structure_residuals[structure].append(residual)

print(f"Computed residues for {len(structure_residuals)} structures across {len(DDD_Boots)} bootstrap iterations")

# Calculate 95% CI for each structure
residual_ci = {}
for structure, residuals in structure_residuals.items():
    residuals_array = np.array(residuals)
    ci_lower = np.percentile(residuals_array, 2.5)
    ci_upper = np.percentile(residuals_array, 97.5)
    median_residual = np.median(residuals_array)
    mean_residual = np.mean(residuals_array)
    
    residual_ci[structure] = {
        'mean': mean_residual,
        'median': median_residual,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'std': np.std(residuals_array)
    }

# Convert to DataFrame for easier viewing
residual_ci_df = pd.DataFrame(residual_ci).T
residual_ci_df = residual_ci_df.sort_values('median', ascending=False)
residual_ci_df.index.name = 'Structure'

print("\n95% CI of bias residues computed for all structures")
print(f"\nTop 10 structures by median residual:")
print(residual_ci_df.head(10))

# Save results
save_dir_ci = "../results/Bootstrap_bias/DDD_ExomeWide/Residual_CI"
os.makedirs(save_dir_ci, exist_ok=True)
residual_ci_df.to_csv(os.path.join(save_dir_ci, "DDD_ExomeWide.Residual_CI_95.csv"))
print(f"\nResults saved to: {save_dir_ci}/DDD_ExomeWide.Residual_CI_95.csv")

In [ ]:
# Load Circuit_STRs if available
try:
    GENCIC = pd.read_csv('../results/GENCIC_MouseSTRBias.csv', index_col=0)
    Circuit_STRs = GENCIC[GENCIC["Circuits.46"]==1]["Structure"].values
except:
    print("Warning: Could not load Circuit_STRs, using all structures")
    Circuit_STRs = None

# Compute merged_data2 from observed ASD vs DDD (exclude ASD)
DDD_GW = Fil2Dict(config["gene_sets"]["DDD_293"]["geneweights"])
DDD_STR_Bias = MouseSTR_AvgZ_Weighted(STR_BiasMat, DDD_GW)
DDD_STR_Bias["Region"] = [Anno.get(ct_idx, "Unknown") for ct_idx in DDD_STR_Bias.index.values]

# Exclude ASD genes from DDD
ASD_GW = Fil2Dict(ProjDIR+"dat/Genetics/GeneWeights_DN/Spark_Meta_EWS.GeneWeight.DN.gw")
ASD_GENES = list(ASD_GW.keys())
DDD_GW_filt_ASD = {k: v for k, v in DDD_GW.items() if k not in ASD_GENES}
DDD_rmASD_STR_Bias = MouseSTR_AvgZ_Weighted(STR_BiasMat, DDD_GW_filt_ASD)
DDD_rmASD_STR_Bias["Region"] = [Anno.get(ct_idx, "Unknown") for ct_idx in DDD_rmASD_STR_Bias.index.values]

# Merge observed data
merged_data2 = merge_str_bias_datasets(Spark_ASD_STR_Bias, DDD_rmASD_STR_Bias, suffixes=('_ASD', '_DD_ExcludeASD'))

# Filter to circuit structures if available
if Circuit_STRs is not None:
    merged_data_eval = merged_data2[merged_data2.index.isin(Circuit_STRs)]
else:
    merged_data_eval = merged_data2.copy()

def plot_top_residual_structures_with_CI(merged_data, residual_ci_df, top_n=30, top_threshold=40, 
                                         name1="ASD", name2="DD", figsize=(10, 8)):
    """
    Plot brain structures with largest residuals from regression analysis, including 95% CI error bars.
    
    Parameters:
    -----------
    merged_data : DataFrame
        Merged dataset with residual and region information
    residual_ci_df : DataFrame
        DataFrame with CI information (columns: ci_lower, ci_upper, median, mean)
    top_n : int
        Number of top structures to display
    top_threshold : int
        Filter to structures in top N of at least one dataset
    name1, name2 : str
        Names of the two datasets being compared
    figsize : tuple
        Figure size (width, height)
    
    Returns:
    --------
    top_diff : DataFrame
        Top structures with largest residuals
    """
    # Filter to only structures that appear in top threshold of at least one dataset
    top_structures = merged_data[(merged_data[f"Rank_{name1}"] <= top_threshold) | 
                                (merged_data[f"Rank_{name2}"] <= top_threshold)]

    print(f"Total structures in top {top_threshold} of at least one dataset: {len(top_structures)}")

    # Sort by absolute difference for top structures only
    top_structures = top_structures.copy()
    top_structures["ABS_DIFF"] = abs(merged_data[f"residual"])
    top_structures = top_structures.sort_values('ABS_DIFF', ascending=True)

    # Take the top N structures with largest differences from those in top threshold
    top_n = min(top_n, len(top_structures))
    top_diff = top_structures.tail(top_n)

    print(f"Showing top {len(top_diff)} structures with largest differences (from top {top_threshold} filter)")

    # Merge with CI data
    top_diff = top_diff.merge(residual_ci_df[['ci_lower', 'ci_upper', 'median']], 
                             left_index=True, right_index=True, how='left')
    
    # Use observed residual (real data) for bar height, CI for error bars
    top_diff['residual_plot'] = top_diff['residual']  # Use observed residual
    top_diff['ci_lower_plot'] = top_diff['ci_lower'].fillna(top_diff['residual'])
    top_diff['ci_upper_plot'] = top_diff['ci_upper'].fillna(top_diff['residual'])

    # Define regions and colors
    REGIONS_seq = ['Isocortex','Olfactory_areas', 'Cortical_subplate', 
                    'Hippocampus','Amygdala','Striatum', 
                    "Thalamus", "Hypothalamus", "Midbrain", 
                    "Medulla", "Pallidum", "Pons", 
                    "Cerebellum"]
    REG_COR_Dic = dict(zip(REGIONS_seq, ["#268ad5", "#D5DBDB", "#7ac3fa", 
                                        "#2c9d39", "#742eb5", "#ed8921", 
                                        "#e82315", "#E6B0AA", "#f6b26b",  
                                        "#20124d", "#2ECC71", "#D2B4DE", 
                                        "#ffd966", ]))

    # Create publication-quality plot of residuals for top_diff structures
    plt.rcParams.update({'font.size': 12, 'font.family': 'Arial'})
    fig, ax = plt.subplots(figsize=figsize, dpi=300)

    # Sort top_diff by residuals for better visualization
    top_diff_sorted = top_diff.sort_values('ABS_DIFF', ascending=True)

    # Create colors based on region
    colors = [REG_COR_Dic.get(region, '#808080') for region in top_diff_sorted['Region']]

    # Calculate error bar positions
    y_pos = range(len(top_diff_sorted))
    x_vals = top_diff_sorted['residual_plot'].values
    xerr_lower = x_vals - top_diff_sorted['ci_lower_plot'].values
    xerr_upper = top_diff_sorted['ci_upper_plot'].values - x_vals
    
    # Create horizontal bar plot with error bars
    bars = ax.barh(y_pos, x_vals, 
                   color=colors, alpha=0.8, edgecolor='black', linewidth=0.5)
    
    # Add error bars
    ax.errorbar(x_vals, y_pos, 
                xerr=[xerr_lower, xerr_upper],
                fmt='none', ecolor='black', elinewidth=1.5, capsize=3, capthick=1.5, alpha=0.7)

    # Customize the plot with publication-quality styling
    ax.set_yticks(y_pos)
    ax.set_yticklabels([name.replace('_', ' ') for name in top_diff_sorted.index], 
                       fontsize=15, fontweight='normal')
    ax.set_xlabel(f'Residuals ({name1} vs {name2}) with 95% CI', fontsize=14, fontweight='bold')

    # Remove top and right spines for cleaner look
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_linewidth(1)
    ax.spines['bottom'].set_linewidth(1)

    # Add subtle grid
    ax.grid(True, axis='x', alpha=0.3, linestyle='-', linewidth=0.5)
    ax.set_axisbelow(True)

    # Add vertical line at x=0 with better styling
    ax.axvline(x=0, color='black', linestyle='-', alpha=0.7, linewidth=1)

    # Create legend for regions with better styling
    unique_regions = sorted(list(set(top_diff_sorted['Region'])))
    legend_elements = [plt.Rectangle((0,0),1,1, facecolor=REG_COR_Dic.get(region, '#808080'), 
                                    alpha=0.8, edgecolor='black', linewidth=0.5) 
                       for region in unique_regions if region in REG_COR_Dic]
    legend_labels = [region.replace('_', ' ') for region in unique_regions if region in REG_COR_Dic]

    if legend_elements:
        ax.legend(
            legend_elements, legend_labels,
            loc='center left',
            bbox_to_anchor=(0.80, 0.15),
            fontsize=10, 
            frameon=True,
            fancybox=True,
            shadow=True,
            framealpha=0.9
        )

    # Adjust layout and margins
    plt.tight_layout()
    plt.subplots_adjust(left=0.3)  # Make room for structure names
    plt.show()
    
    return top_diff

# Create the plot with 95% CI
top_diff = plot_top_residual_structures_with_CI(merged_data_eval, residual_ci_df, 
                                                top_n=20, top_threshold=40,
                                                name1="ASD", name2="DD_ExcludeASD", 
                                                figsize=(6, 8))

In [ ]:
## LOEUF25 Bootstrap Analysis

# Load gnomad4 constraint data and define LOEUF25 gene set
gnomad4 = pd.read_csv("/home/jw3514/Work/data/gnomad/gnomad.v4.0.constraint_metrics.tsv", sep="\t")
search_text = 'ENST'
gnomad4 = gnomad4[(gnomad4["transcript"].str.contains(search_text))]
gnomad4 = gnomad4[gnomad4["mane_select"]==True]

# Convert gene symbols to Entrez IDs
for i, row in gnomad4.iterrows():
    symbol = row["gene"]
    gnomad4.loc[i, "Entrez"] = int(GeneSymbol2Entrez.get(symbol, 0))

# Take subset where lof.oe_ci.upper is in the bottom 25% (most constrained)
bottom_25_percent_threshold = gnomad4["lof.oe_ci.upper"].quantile(0.25)
gnomad4_bottom25 = gnomad4[gnomad4["lof.oe_ci.upper"] <= bottom_25_percent_threshold]
columns_to_keep_g4 = ["Entrez", "gene", "lof.pLI", "lof.z_score", "lof.oe_ci.upper"]
gnomad4_bottom25 = gnomad4_bottom25[columns_to_keep_g4]
gnomad4_bottom25 = gnomad4_bottom25.sort_values(by="lof.oe_ci.upper", ascending=True)

# Make sure Entrez is int and exclude rows with Entrez = 0
gnomad4_bottom25["Entrez"] = gnomad4_bottom25["Entrez"].astype(int)
gnomad4_bottom25 = gnomad4_bottom25[gnomad4_bottom25["Entrez"] != 0]

# Get LOEUF25 gene list (Entrez IDs)
LOEUF25_genes = gnomad4_bottom25["Entrez"].unique().tolist()
print(f"LOEUF25 gene set: {len(LOEUF25_genes)} genes")
print(f"Bottom 25% threshold: {bottom_25_percent_threshold:.4f}")


In [ ]:
def bootstrap_genes_from_set(gene_set, n_boot=1000, n_genes=None, rng=None):
    """
    Bootstrap sample genes from a given gene set.
    
    Parameters
    ----------
    gene_set : list
        List of Entrez gene IDs to sample from
    n_boot : int
        Number of bootstrap replicates
    n_genes : int, optional
        Number of genes to sample per bootstrap. If None, uses length of gene_set
    rng : np.random.Generator, optional
        Numpy random generator for reproducibility
    
    Returns
    -------
    boot_gene_sets : list of lists
        List of bootstrap gene sets, each containing sampled Entrez IDs
    """
    if rng is None:
        rng = np.random.default_rng(42)
    
    if n_genes is None:
        n_genes = len(gene_set)
    
    gene_set_array = np.array(gene_set)
    boot_gene_sets = []
    
    for i in range(1, n_boot + 1):
        # Sample n_genes with replacement from gene_set
        boot_genes = rng.choice(gene_set_array, size=n_genes, replace=True)
        boot_gene_sets.append(boot_genes.tolist())
    
    return boot_gene_sets

# Bootstrap genes from LOEUF25 gene set
print(f"Bootstrapping {len(LOEUF25_genes)} genes from LOEUF25 gene set...")
LOEUF25_boot_gene_sets = bootstrap_genes_from_set(LOEUF25_genes, n_boot=1000, rng=np.random.default_rng(42))
print(f"Created {len(LOEUF25_boot_gene_sets)} bootstrap replicates")


In [ ]:
# Process LOEUF25 bootstrap iterations in parallel
def process_LOEUF25_bootstrap_iter(args):
    """Worker function to process a single LOEUF25 bootstrap iteration"""
    i, boot_genes, save_dir, str_bias_mat = args
    # Create gene weights dictionary (equal weights of 1)
    boot_gw = {gene: 1.0 for gene in boot_genes}
    boot_bias = MouseSTR_AvgZ_Weighted(str_bias_mat, boot_gw)
    boot_bias.to_csv(os.path.join(save_dir, f"LOEUF25.GeneWeight.boot{i}.csv"))
    return i, boot_bias

save_dir_LOEUF25 = "../results/Bootstrap_bias/LOEUF25/Weighted_Resampling"
os.makedirs(save_dir_LOEUF25, exist_ok=True)

# Prepare arguments for parallel processing
n_workers = mp.cpu_count()
args_list_LOEUF25 = [(i, boot_genes, save_dir_LOEUF25, STR_BiasMat) 
                     for i, boot_genes in enumerate(LOEUF25_boot_gene_sets)]

# Process in parallel
print(f"Using {n_workers} workers to compute LOEUF25 bootstrap bias...")
LOEUF25_boot_bias_list = [None] * len(LOEUF25_boot_gene_sets)

with ProcessPoolExecutor(max_workers=n_workers) as executor:
    future_to_idx = {executor.submit(process_LOEUF25_bootstrap_iter, args): args[0] 
                     for args in args_list_LOEUF25}
    
    completed = 0
    for future in as_completed(future_to_idx):
        i, boot_bias = future.result()
        LOEUF25_boot_bias_list[i] = boot_bias
        completed += 1
        if completed % 100 == 0:
            print(f"Completed {completed}/{len(LOEUF25_boot_gene_sets)} LOEUF25 bootstrap iterations")

print(f"Completed all {len(LOEUF25_boot_bias_list)} LOEUF25 bootstrap iterations")


In [ ]:
# Compute residues for LOEUF25 bootstrap vs ASD bootstrap (parallelized)
print("Computing residues for LOEUF25 bootstrap vs ASD observed...")

def compute_residuals_LOEUF25_vs_ASD(args):
    """Worker function to compute residuals for a single LOEUF25 bootstrap iteration vs ASD"""
    i, loeuf25_boot, asd_bias = args
    merged = merge_str_bias_datasets(asd_bias, loeuf25_boot, suffixes=('_ASD', '_LOEUF25'))
    return i, merged['residual'].to_dict()

# Prepare arguments for parallel processing
args_list_residuals_LOEUF25 = [(i, loeuf25_boot, Spark_ASD_STR_Bias) 
                               for i, loeuf25_boot in enumerate(LOEUF25_boot_bias_list)]

# Process in parallel
print(f"Using {n_workers} workers to compute LOEUF25 residuals...")
LOEUF25_residual_results = [None] * len(LOEUF25_boot_bias_list)

with ProcessPoolExecutor(max_workers=n_workers) as executor:
    future_to_idx = {executor.submit(compute_residuals_LOEUF25_vs_ASD, args): args[0] 
                     for args in args_list_residuals_LOEUF25}
    
    completed = 0
    for future in as_completed(future_to_idx):
        i, residuals_dict = future.result()
        LOEUF25_residual_results[i] = residuals_dict
        completed += 1
        if completed % 100 == 0:
            print(f"Completed {completed}/{len(LOEUF25_boot_bias_list)} LOEUF25 residual computations")

# Collect all residues for each structure
LOEUF25_structure_residuals = {}
for residuals_dict in LOEUF25_residual_results:
    for structure, residual in residuals_dict.items():
        if structure not in LOEUF25_structure_residuals:
            LOEUF25_structure_residuals[structure] = []
        LOEUF25_structure_residuals[structure].append(residual)

print(f"Computed residues for {len(LOEUF25_structure_residuals)} structures across {len(LOEUF25_boot_bias_list)} bootstrap iterations")

# Calculate 95% CI for each structure
LOEUF25_residual_ci = {}
for structure, residuals in LOEUF25_structure_residuals.items():
    residuals_array = np.array(residuals)
    ci_lower = np.percentile(residuals_array, 2.5)
    ci_upper = np.percentile(residuals_array, 97.5)
    median_residual = np.median(residuals_array)
    mean_residual = np.mean(residuals_array)
    
    LOEUF25_residual_ci[structure] = {
        'mean': mean_residual,
        'median': median_residual,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'std': np.std(residuals_array)
    }

# Convert to DataFrame
LOEUF25_residual_ci_df = pd.DataFrame(LOEUF25_residual_ci).T
LOEUF25_residual_ci_df = LOEUF25_residual_ci_df.sort_values('median', ascending=False)
LOEUF25_residual_ci_df.index.name = 'Structure'

print("\n95% CI of LOEUF25 bias residues computed for all structures")
print(f"\nTop 10 structures by median residual:")
print(LOEUF25_residual_ci_df.head(10))

# Save results
save_dir_LOEUF25_ci = "../results/Bootstrap_bias/LOEUF25/Residual_CI"
os.makedirs(save_dir_LOEUF25_ci, exist_ok=True)
LOEUF25_residual_ci_df.to_csv(os.path.join(save_dir_LOEUF25_ci, "LOEUF25.Residual_CI_95.csv"))
print(f"\nResults saved to: {save_dir_LOEUF25_ci}/LOEUF25.Residual_CI_95.csv")


In [ ]:
# Compute observed LOEUF25 bias for comparison
# IMPORTANT: Match DDD.ipynb exactly - use all Entrez IDs from gnomad4_bottom25 (not just unique)
# In DDD.ipynb: constraint_gw_top_LOEUF25 = dict(zip(gnomad4_bottom25["Entrez"], [1]*len(gnomad4_bottom25)))
# This ensures we get the exact same gene weights as DDD.ipynb
LOEUF25_gw_observed = dict(zip(gnomad4_bottom25["Entrez"], [1]*len(gnomad4_bottom25)))
LOEUF25_STR_Bias_observed = MouseSTR_AvgZ_Weighted(STR_BiasMat, LOEUF25_gw_observed)
LOEUF25_STR_Bias_observed["Region"] = [Anno.get(ct_idx, "Unknown") for ct_idx in LOEUF25_STR_Bias_observed.index.values]

# Merge observed data for LOEUF25 vs ASD
merged_data_LOEUF25_observed = merge_str_bias_datasets(Spark_ASD_STR_Bias, LOEUF25_STR_Bias_observed, 
                                                       suffixes=('_ASD', '_LOEUF25'))

# Filter to circuit structures if available
if Circuit_STRs is not None:
    merged_data_LOEUF25_eval = merged_data_LOEUF25_observed[merged_data_LOEUF25_observed.index.isin(Circuit_STRs)]
else:
    merged_data_LOEUF25_eval = merged_data_LOEUF25_observed.copy()

# Create the plot with 95% CI
top_diff_LOEUF25 = plot_top_residual_structures_with_CI(merged_data_LOEUF25_eval, LOEUF25_residual_ci_df, 
                                                        top_n=20, top_threshold=40,
                                                        name1="ASD", name2="LOEUF25", 
                                                        figsize=(6, 8))
